# Lector Objetos de Tweets


### 1. Importing Required Libraries

In [1]:
import zipfile
from pandas.io.json import json_normalize
import pandas as pd

### 2. Data

#### 2.1. UnZip

Primero extraemos los datos de la carpeta .zip para poder leer los datos

In [2]:
DIR_ZIP = './data/archive.zip'
DIR_UNZIP = './data'

with zipfile.ZipFile(DIR_ZIP, 'r') as zip_ref:
    zip_ref.extractall(DIR_UNZIP)

#### 2.2. Data Lecture

In [3]:
FILE_DIR = DIR_UNZIP + '/farmers-protest-tweets-2021-03-5.json'

# Read JSON file containing tweets data and removce tweets not in English

raw_tweets = pd.read_json(FILE_DIR, lines=True)
raw_tweets = raw_tweets[raw_tweets['lang']=='en']
print("Shape: ", raw_tweets.shape)
raw_tweets.head(5)

Shape:  (417511, 21)


,url,date,content,renderedContent,id,user,outlinks,tcooutlinks,replyCount,retweetCount,...,quoteCount,conversationId,lang,source,sourceUrl,sourceLabel,media,retweetedTweet,quotedTweet,mentionedUsers
0,https://twitter.com/ShashiRajbhar6/status/1376...,2021-03-30 03:33:46+00:00,Support 👇\n\n#FarmersProtest,Support 👇\n\n#FarmersProtest,1376739399593910273,"{'username': 'ShashiRajbhar6', 'displayname': ...",[],[],0,0,...,0,1376739399593910273,en,"<a href=""http://twitter.com/download/android"" ...",http://twitter.com/download/android,Twitter for Android,None,NaN,None,None
1,https://twitter.com/kaursuk06272818/status/137...,2021-03-30 03:33:23+00:00,Supporting farmers means supporting our countr...,Supporting farmers means supporting our countr...,1376739306287427584,"{'username': 'kaursuk06272818', 'displayname':...",[],[],0,0,...,0,1376739306287427584,en,"<a href=""http://twitter.com/download/android"" ...",http://twitter.com/download/android,Twitter for Android,[{'previewUrl': 'https://pbs.twimg.com/media/E...,NaN,None,None
2,https://twitter.com/kaursuk06272818/status/137...,2021-03-30 03:31:00+00:00,Support farmers if you are related to food #St...,Support farmers if you are related to food #St...,1376738704128020488,"{'username': 'kaursuk06272818', 'displayname':...",[],[],0,0,...,0,1376738704128020488,en,"<a href=""http://twitter.com/download/android"" ...",http://twitter.com/download/android,Twitter for Android,[{'previewUrl': 'https://pbs.twimg.com/media/E...,NaN,None,None
3,https://twitter.com/SukhdevSingh_/status/13767...,2021-03-30 03:30:45+00:00,#StopHateAgainstFarmers support #FarmersProtes...,#StopHateAgainstFarmers support #FarmersProtes...,1376738640542400518,"{'username': 'SukhdevSingh_', 'displayname': '...",[],[],0,1,...,0,1376738640542400518,en,"<a href=""http://twitter.com/download/android"" ...",http://twitter.com/download/android,Twitter for Android,None,NaN,None,None
4,https://twitter.com/Davidmu66668113/status/137...,2021-03-30 03:30:30+00:00,"You hate farmers I hate you, \nif you love the...","You hate farmers I hate you, \nif you love the...",1376738579171344386,"{'username': 'Davidmu66668113', 'displayname':...",[],[],0,0,...,0,1376738579171344386,en,"<a href=""http://twitter.com/download/android"" ...",http://twitter.com/download/android,Twitter for Android,None,NaN,None,None


#### 2.3. Pre-process Data

In [59]:
Tweets = raw_tweets[['date', 'content', 'renderedContent', 'user', 'retweetCount']]

In [61]:
def focusOnUserAttr(col, attr):
    def func(df):
        df[col] = df[col].apply(lambda x: x[attr])
        return df 
    return func

focusOnUserAttr('user', 'username')(Tweets)

<ipython-input-61-03ec17165372>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[col] = df[col].apply(lambda x: x[attr])


,date,content,renderedContent,user,retweetCount
0,2021-03-30 03:33:46+00:00,Support 👇\n\n#FarmersProtest,Support 👇\n\n#FarmersProtest,ShashiRajbhar6,0
1,2021-03-30 03:33:23+00:00,Supporting farmers means supporting our countr...,Supporting farmers means supporting our countr...,kaursuk06272818,0
2,2021-03-30 03:31:00+00:00,Support farmers if you are related to food #St...,Support farmers if you are related to food #St...,kaursuk06272818,0
3,2021-03-30 03:30:45+00:00,#StopHateAgainstFarmers support #FarmersProtes...,#StopHateAgainstFarmers support #FarmersProtes...,SukhdevSingh_,1
4,2021-03-30 03:30:30+00:00,"You hate farmers I hate you, \nif you love the...","You hate farmers I hate you, \nif you love the...",Davidmu66668113,0
...,...,...,...,...,...
419502,2021-02-01 00:04:26+00:00,We are calling on leaders to speak up against ...,We are calling on leaders to speak up against ...,TeamSOOUDA,3
419503,2021-02-01 00:04:16+00:00,Power💪🏻 of panchayat but our government still ...,Power💪🏻 of panchayat but our government still ...,VibhaBurman,0
419504,2021-02-01 00:01:26+00:00,@AmrinderSingh__ @CNN @cnni I’m coming to term...,@AmrinderSingh__ @CNN @cnni I’m coming to term...,NavJammu,0
419505,2021-02-01 00:00:42+00:00,What about the peaceful protests in Delhi........,What about the peaceful protests in Delhi........,Atwal72,0


### 3. Ranking

In [62]:
def topTen(func, df):
    return func(df).head(10)

In [63]:
def changeDF(func, df):
    return func(df)

#### 3.1. Filters

##### 3.1.a More Re-Tweeted

In [64]:
def moreReTweeted(df):
    return df.sort_values(by='retweetCount', ascending=False)

In [74]:
topTen(moreReTweeted, Tweets)[['content', 'retweetCount']]

,content,retweetCount
408128,why aren’t we talking about this?! #FarmersPro...,315547
395142,We stand in solidarity with the #FarmersProtes...,103957
266196,I still #StandWithFarmers and support their pe...,67694
366579,"“Paid actors,” huh? Quite the casting director...",35921
372793,What in the human rights violations is going o...,26972
314192,"Happy to share that I’ve donated $10,000 to pr...",23251
215034,There has been much social media coverage arou...,20132
398011,Farmers feed the world. Fight for them. Protec...,18744
325261,Should be talking about this! #FarmersProtest\...,17368
163689,To all of my influencer/celeb friends- read up...,15677


##### 3.1.b More Tweeted

In [68]:
def countTweetsPerUser(df):
    return df.groupby('user').count().sort_values(by='content',ascending=False)

In [70]:
topTen(countTweetsPerUser,Tweets)['content']

user
harjot_tweeting    7134
tasveersandhu      2088
shells_n_petals    1991
jot__b             1841
rebelpacifist      1803
rumsomal           1722
Iamjazzie96        1491
Jass_k_G           1458
DigitalKisanBot    1453
z_khalique007      1446
Name: content, dtype: int64